# En el siguiente notebook vamos a trabajar solo con la simulación, considerando el equivalente  a las 350  estrellas de los datos reales. Haremos un subsampleo considerando la mascara del plano galáctico, y diferentes 'seeds'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import ascii
from astropy.coordinates import SkyCoord, CartesianRepresentation
from astropy.wcs import WCS
import gala.coordinates as gc
import healpy as hp
from pylab import cm
import matplotlib as mpl
from matplotlib.patches import Circle, PathPatch, Ellipse
from healpy.newvisufunc import projview, newprojplot
from mpl_toolkits.mplot3d import Axes3D
from astropy.coordinates import cartesian_to_spherical
from matplotlib.lines import Line2D
from matplotlib.path import Path
from matplotlib.legend_handler import HandlerTuple
import datosCM
import pymaster as nmt

from matplotlib.patches import Rectangle

In [ ]:
from astropy.coordinates import SkyCoord

def cat_to_hpx(lon, lat, nside, radec=False):   #radec=False: si vamos de cartesianas a esfericas galacticas
    """
    Convierte un catálogo a un mapa HEALPix de densidad de número, es decir,
    el número de estrellas por grado cuadrado de cielo.

    Parámetros
    ----------
    lon, lat : (arreglo, arreglo)
        Coordenadas de las fuentes en grados.
        Si radec=True, asume que la entrada está en el sistema ICRS,
        De lo contrario, asume que la entrada está en latitud y longitud galáctica.

    nside : int
        Número HEALPix nside del mapa de destino, define el número de píxeles.

    radec : bool
        Cambio entre Ra/Dec y l/b (galáctico) como sistema de coordenadas de entrada.

    Return
    ------
    hpx_map : arreglo
        Mapa HEALPix de los recuentos de número del catálogo en coordenadas galácticas
    """
    npix = hp.nside2npix(nside)

    if radec:
        eq = SkyCoord(lon, lat, unit='deg')
        l, b = eq.galactic.l.value, eq.galactic.b.value
    else:
        l, b = lon, lat

    # OJO ACA  

    theta = np.radians(90. - b)                 # latitud en radianes
    phi = np.radians(l)               # longitud en radianes        
    indices = hp.ang2pix(nside, theta, phi, lonlat=False)  


    indx, counts = np.unique(indices, return_counts=True)

    # llenar el mapa de cielo completo
    hpx_map = np.zeros(npix, dtype=int)
    hpx_map[indx] = counts

    return hpx_map


In [ ]:
fwhm2sigma = lambda fwhm: fwhm / np.sqrt(8*np.log(2))


# Parámetros a usar 

In [ ]:
nside=128
lmax=3*nside-1
#lmax=47
espesor=5 #kpc
radio=70 #kpc

radio_min=radio-espesor/2
radio_max=radio+espesor/2

# Leyendo los datos de la simulación 

In [ ]:
#lectura de datos

simu = pd.read_csv('rand_mwlmc5b0_110.txt', sep=' ',names=['x','y','z','vx', 'vy', 'vz'])

#simu = pd.read_csv('/home/marz/Dropbox/PlanAB/gkrr/Wake/notebooks wake simu/rand_mwlmc5b0_110.txt', sep=' ',
#                  names=['x','y','z','vx', 'vy', 'vz'])

#mc = pd.read_csv('/content/drive/MyDrive/rand_mwb1_110.txt', sep=' ',
                  #names=['x','y','z','vx', 'vy', 'vz'])
#pd.DataFrame(mc)

# con x,y,z en kpc y vx, vy, vz en km/s

simu

## Cálculo de las coordenadas $r,\theta, \phi$ con ```astropy```

In [ ]:
from astropy.coordinates import cartesian_to_spherical

x = simu['x'].values #* u.Mpc
y = simu['y'].values #* u.Mpc
z = simu['z'].values #* u.Mpc

#
spherical_coords = cartesian_to_spherical(x, y, z)

#
r = spherical_coords[0]#.to(u.Mpc, equivalencies=u.dimensionless_angles()).value
theta = spherical_coords[1].to(u.rad).value
phi = spherical_coords[2].to(u.rad).value
phi2 = spherical_coords[2].to(u.degree).value

# Añadir los resultados de nuevo al DataFrame
simu['R'] = r                #kpc
simu['theta'] = theta        #rad
simu['phi'] = phi            #rad
simu['phi2'] = phi2         # en grados
simu['cotheta'] = np.pi/2 - theta          #rad

simu

# Primero sobre toda la simuladión con máscara pero sin condición de distancia

In [ ]:
# Convertir coordenadas esféricas a galácticas

l_deg = np.degrees(phi)
b_deg = np.degrees(theta)

# OJO AQUI. cat_to_hpx espera el mapa en grados, no en radianes
mapa_simu = cat_to_hpx(l_deg, b_deg, nside=nside, radec=False)
mapa_simu_mean = mapa_simu.mean()
#sobredensidad (no suavizado)
sobre_simu = (mapa_simu - mapa_simu_mean) / mapa_simu_mean


In [ ]:
l_sim = coord.Angle(simu['phi'].values*180/np.pi*u.degree)
l_sim = l_sim.wrap_at(180*u.degree)
b_sim = coord.Angle(simu['theta'].values*180/np.pi*u.degree)

simulacion= SkyCoord(l=l_sim, b=b_sim, frame='galactic')
ipix_simulacion = hp.ang2pix(nside, np.pi/2- simulacion.galactic.b.radian, simulacion.galactic.l.radian)

In [ ]:
#sobredensidad (no suavizado)
l = simulacion.galactic.l.degree
b = simulacion.galactic.b.degree
mapa_simulacion = cat_to_hpx(l, b, nside=nside, radec=False)
mapa_simulacion_mean = mapa_simulacion.mean()
mapa_simulacion = (mapa_simulacion - mapa_simulacion_mean) / mapa_simulacion_mean

# Suavizar el mapa SOLO para visualización
mapa_simulacion_smooth= hp.sphtfunc.smoothing(mapa_simulacion, sigma=np.radians(fwhm2sigma(30)))

# Crear máscara del plano galáctico |b| < 10°
xg, yg = np.meshgrid(
    np.pi/2 - np.linspace(np.radians(-10), np.radians(10), 10000),
    np.linspace(0, 2*np.pi, 10000)
)
mask = hp.ang2pix(nside, xg, yg)

# Crear copia enmascarada del mapa suavizado solo
mapa_simulacion_smooth_masked = mapa_simulacion_smooth.copy()
mapa_simulacion_smooth_masked[mask] = 0

# Crear copia enmascarada del mapa NO suavizado (para calculos  de alm y cls)
mapa_simulacion_masked = mapa_simulacion.copy()
mapa_simulacion_masked[mask] = 0
# 
fig = plt.figure(figsize=(9, 5))
ax = fig.add_subplot()

mmin, mmax = np.min(mapa_simulacion_smooth_masked[np.isfinite(mapa_simulacion_smooth_masked)]), np.max(mapa_simulacion_smooth_masked[np.isfinite(mapa_simulacion_smooth_masked)])

hp.mollview(mapa_simulacion_smooth_masked, title='Mapa de sobredensidad simulación', 
            unit=r"$\Delta \rho /\ \bar{\rho}$",  cbar=True, flip='astro', hold=True) 

plt.tight_layout()
plt.savefig('mapa_simulacion_smooth_masked_'+str(radio)+'.pdf')
plt.show()


# Para los Cls

In [ ]:
cl_simu=hp.anafast(mapa_simulacion_masked, alm=True,lmax=lmax, pol=False)

In [ ]:
len(cl_simu[0])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,4))
fig.suptitle('Simulation: Power Spectra, all stars')

ells = np.arange(lmax + 1)


ax[0].plot(ells, cl_simu[0][:lmax + 1], 
         label="$C_{\ell}$ Simu", marker="o", linestyle="--", color='blue', alpha=0.7)
ax[0].set_xlabel('$\ell$')
ax[0].set_xlim(-1,15)
ax[0].set_ylabel('$C_{\ell}$')
ax[0].set_ylim(-0.001, 0.08)
ax[0].legend()

ax[1].plot(ells, (2*ells +1)*cl_simu[0][:lmax + 1], 
         label="$(2 \ell +1) C_{\ell}$ Simu ", marker="o", linestyle="--", color='red', alpha=0.7)
ax[1].set_xlabel('$\ell$')
ax[1].set_xlim(-1,15)
ax[1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[1].set_ylim(-0.005, 0.20)
ax[1].legend()

#plt.savefig('cl_sobre_simu_'+str(radio)+'.pdf') 
plt.tight_layout()
plt.show()



### Condición para un anillo de 70 kpc $\pm$ 5 kpc 

In [ ]:
simu['vr'] = np.sin(simu['cotheta']) * np.cos(simu['phi'])* simu['vx'] + np.sin(simu['cotheta']) * np.sin(simu['phi']) * simu['vy'] + np.cos(simu['cotheta']) * simu['vz']
simu['vtheta'] = np.cos(simu['cotheta'])* np.cos(simu['phi']) * simu['vx'] + np.sin(simu['phi']) * np.cos(simu['cotheta']) * simu['vy'] - np.sin(simu['cotheta']) * simu['vz']
simu['vphi'] =  ( np.sin(simu['phi'])) * simu['vx'] -  np.cos(simu['phi']) * simu['vy']

simu_condicion70=simu[(simu['R']<radio_max) & (simu['R']>radio_min)]
simu_condicion70.reset_index(drop=True, inplace=True)
print(len(simu_condicion70))

simu_condicion70

In [ ]:
# Convertir coordenadas esféricas a galácticas

l_deg = np.degrees(phi)
b_deg = np.degrees(theta)

# OJO AQUI. cat_to_hpx espera el mapa en grados, no en radianes
mapa_simu = cat_to_hpx(l_deg, b_deg, nside=nside, radec=False)
mapa_simu_mean = mapa_simu.mean()
#sobredensidad (no suavizado)
sobre70_simu = (mapa_simu - mapa_simu_mean) / mapa_simu_mean




In [ ]:
l_sim = coord.Angle(simu_condicion70['phi'].values*180/np.pi*u.degree)
l_sim = l_sim.wrap_at(180*u.degree)
b_sim = coord.Angle(simu_condicion70['theta'].values*180/np.pi*u.degree)

simulacion= SkyCoord(l=l_sim, b=b_sim, frame='galactic')
ipix_simulacion = hp.ang2pix(nside, np.pi/2- simulacion.galactic.b.radian, simulacion.galactic.l.radian)

In [ ]:
#sobredensidad (no suavizado)
l = simulacion.galactic.l.degree
b = simulacion.galactic.b.degree
mapa_simulacion = cat_to_hpx(l, b, nside=nside, radec=False)
mapa_simulacion_mean = mapa_simulacion.mean()
mapa_simulacion = (mapa_simulacion - mapa_simulacion_mean) / mapa_simulacion_mean

# Suavizar el mapa SOLO para visualización
mapa_simulacion_smooth= hp.sphtfunc.smoothing(mapa_simulacion, sigma=np.radians(fwhm2sigma(30)))

# Crear máscara del plano galáctico |b| < 10°
xg, yg = np.meshgrid(
    np.pi/2 - np.linspace(np.radians(-10), np.radians(10), 10000),
    np.linspace(0, 2*np.pi, 10000)
)
mask = hp.ang2pix(nside, xg, yg)

# Crear copia enmascarada del mapa suavizado solo
mapa_simulacion_smooth_masked = mapa_simulacion_smooth.copy()
mapa_simulacion_smooth_masked[mask] = 0

# Crear copia enmascarada del mapa NO suavizado (para calculos  de alm y cls)
mapa_simulacion_masked = mapa_simulacion.copy()
mapa_simulacion_masked[mask] = 0
# 
fig = plt.figure(figsize=(9, 5))
ax = fig.add_subplot()

mmin, mmax = np.min(mapa_simulacion_smooth_masked[np.isfinite(mapa_simulacion_smooth_masked)]), np.max(mapa_simulacion_smooth_masked[np.isfinite(mapa_simulacion_smooth_masked)])

hp.mollview(mapa_simulacion_smooth_masked, title='Mapa de sobredensidad simulación', 
            unit=r"$\Delta \rho /\ \bar{\rho}$",  cbar=True, flip='astro', hold=True) 

plt.tight_layout()
plt.savefig('mapa_simulacion_smooth_masked_'+str(radio)+'.pdf')
plt.show()


In [ ]:
cl_simu70 = hp.anafast(mapa_simulacion_masked, lmax=lmax, alm=True, pol=False)
print(len(cl_simu70[0]))

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,4))
fig.suptitle('Simulation: Power Spectra,  ('+str(radio)+')')

ells = np.arange(lmax + 1)


ax[0].plot(ells, cl_simu70[0][:lmax + 1], 
         label="$C_{\ell}$ Simu", marker="o", linestyle="--", color='blue', alpha=0.7)
ax[0].set_xlabel('$\ell$')
ax[0].set_xlim(-1,15)
ax[0].set_ylabel('$C_{\ell}$')
ax[0].set_ylim(-0.001, 0.04)
ax[0].legend()

ax[1].plot(ells, (2*ells +1)*cl_simu70[0][:lmax + 1], 
         label="$(2 \ell +1) C_{\ell}$ Simu ", marker="o", linestyle="--", color='red', alpha=0.7)
ax[1].set_xlabel('$\ell$')
ax[1].set_xlim(-1,15)
ax[1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[1].set_ylim(-0.005, 0.14)
ax[1].legend()

#plt.savefig('cl_sobre_simu_'+str(radio)+'.pdf') 
plt.tight_layout()
plt.show()



# Para el muestreo aleatorio

In [ ]:
simu_sample = simu_condicion70.sample(n=2150,random_state=250)
simu_sample.reset_index(drop=True, inplace=True)
simu_sample

In [ ]:
l_sim = coord.Angle(simu_sample['phi'].values*180/np.pi*u.degree)
l_sim = l_sim.wrap_at(180*u.degree)
b_sim = coord.Angle(simu_sample['theta'].values*180/np.pi*u.degree)

simulacion= SkyCoord(l=l_sim, b=b_sim, frame='galactic')
ipix_simulacion = hp.ang2pix(nside, np.pi/2- simulacion.galactic.b.radian, simulacion.galactic.l.radian)

## Usando la mascara. Sobredensidad:

In [ ]:
#sobredensidad (no suavizado)
l = simulacion.galactic.l.degree
b = simulacion.galactic.b.degree
mapa_simulacion = cat_to_hpx(l, b, nside=nside, radec=False)
mapa_simulacion_mean = mapa_simulacion.mean()
mapa_simulacion = (mapa_simulacion - mapa_simulacion_mean) / mapa_simulacion_mean

# Suavizar el mapa SOLO para visualización
mapa_simulacion_smooth= hp.sphtfunc.smoothing(mapa_simulacion, sigma=np.radians(fwhm2sigma(30)))

# Crear máscara del plano galáctico |b| < 10°
xg, yg = np.meshgrid(
    np.pi/2 - np.linspace(np.radians(-10), np.radians(10), 10000),
    np.linspace(0, 2*np.pi, 10000)
)
mask = hp.ang2pix(nside, xg, yg)

# Crear copia enmascarada del mapa suavizado solo
mapa_simulacion_smooth_masked = mapa_simulacion_smooth.copy()
mapa_simulacion_smooth_masked[mask] = 0

# Crear copia enmascarada del mapa NO suavizado (para calculos  de alm y cls)
mapa_simulacion_masked = mapa_simulacion.copy()
mapa_simulacion_masked[mask] = 0
# 
fig = plt.figure(figsize=(9, 5))
ax = fig.add_subplot()

mmin, mmax = np.min(mapa_simulacion_smooth_masked[np.isfinite(mapa_simulacion_smooth_masked)]), np.max(mapa_simulacion_smooth_masked[np.isfinite(mapa_simulacion_smooth_masked)])

hp.mollview(mapa_simulacion_smooth_masked, title='Mapa de sobredensidad simulación', 
            unit=r"$\Delta \rho /\ \bar{\rho}$",  cbar=True, flip='astro', hold=True) 

plt.tight_layout()
plt.savefig('mapa_simulacion_smooth_masked_'+str(radio)+'.pdf')
plt.show()


In [ ]:
mask_fuera_plano = np.abs(b) >= 10
print(f"Estrellas fuera del plano: {np.sum(mask_fuera_plano)} / {len(simulacion)}")
print("estrellas disponibles: ", len(simulacion) - np.sum(mask_fuera_plano))

## Para un solo submuestreo

In [ ]:
cl_simu70_sub = hp.anafast(mapa_simulacion_masked, alm=True, lmax=lmax)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,4))
fig.suptitle('Simulation: Power Spectra,  ('+str(radio)+') kpc')

ells = np.arange(lmax + 1)


ax[0].plot(ells, cl_simu70_sub[0][:lmax + 1], 
         label="$C_{\ell}$ Simu", marker="o", linestyle="--", color='blue', alpha=0.7)
ax[0].set_xlabel('$\ell$')
ax[0].set_xlim(0, 15)
ax[0].set_ylabel('$C_{\ell}$')
ax[0].set_ylim(0, 0.04)

ax[1].plot(ells, (2*ells +1)*cl_simu70_sub[0][:lmax + 1], 
         label="$(2 \ell +1) C_{\ell}$ Simu ", marker="o", linestyle="--", color='red', alpha=0.7)
ax[1].set_xlabel('$\ell$')
ax[1].set_xlim(0, 15)
ax[1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[1].set_ylim(0, 0.15)
ax[1].legend()

plt.tight_layout()

plt.show()



In [ ]:
N = 1000  # numero de submuestreos
n_muestreo = 2150

cls_all = []

# mascara del plano 
xg, yg = np.meshgrid(
    np.pi/2 - np.linspace(np.radians(-10), np.radians(10), 10000),
    np.linspace(0, 2*np.pi, 10000)
)
mask_pix = hp.ang2pix(nside, xg, yg)

for seed in range(N):
    # para el submuestreo aleatorio
    simu_sample = simu_condicion70.sample(n=n_muestreo, random_state=seed).reset_index(drop=True)

    # de radianes a grados l y b 
    l_sim = coord.Angle(simu_sample['phi'].values * 180/np.pi * u.deg).wrap_at(180 * u.deg)
    b_sim = coord.Angle(simu_sample['theta'].values * 180/np.pi * u.deg)
    simulacion = SkyCoord(l=l_sim, b=b_sim, frame='galactic')
    l = simulacion.galactic.l.degree
    b = simulacion.galactic.b.degree

    # mapa de sobredensidad
    mapa = cat_to_hpx(l, b, nside=nside, radec=False)
    mapa_mean = mapa.mean()
    mapa = (mapa - mapa_mean) / mapa_mean

    # usando la mascarA
    mapa_masked = mapa.copy()
    mapa_masked[mask_pix] = 0

    # Calcular Cl y alm
    cl, alm = hp.anafast(mapa_masked, alm=True, lmax=lmax)
    cls_all.append(cl)

# Convertir a array y calcular estadística
cls_all = np.array(cls_all)
ell = np.arange(cls_all.shape[1])
cl_mean = np.mean(cls_all, axis=0)
cl_std = np.std(cls_all, axis=0)


In [ ]:
print(cls_all.shape)
print(ell.shape)
print(cl_mean.shape)
print(cl_std.shape)

In [ ]:

fig, ax = plt.subplots(1,2, figsize=(8,4))
fig.suptitle('Simulation: Power Spectra,  ('+str(radio)+') kpc')

ells = np.arange(lmax + 1)

ax[0].plot(ells, cl_mean, marker="o",  color='magenta', alpha=0.6,lw=2.5,label=r"$\langle C_{\ell} \rangle $ subsample N="+str(N))
ax[0].fill_between(ell, cl_mean - cl_std, cl_mean + cl_std, color='grey', alpha=0.2, label=r"$\pm 1\sigma$")
#ax[0].bar(ell, cl_mean, yerr=cl_std, align='center', alpha=0.1, ecolor='black', capsize=1)
# Solo los puntos con barra de error
ax[0].errorbar(ell, cl_mean, yerr=cl_std, fmt='o-', color='orange',alpha=0.2,
             ecolor='black', capsize=3, label=r"")

ax[0].plot(ells, cl_simu70_sub[0][:lmax + 1], 
         label="$C_{\ell}$ Simu", marker="o", linestyle="--", color='blue', alpha=0.7)

ax[0].set_xlabel('$\ell$')
ax[0].set_xlim(-1, 20)
ax[0].set_ylabel('$C_{\ell}$')
ax[0].set_ylim(-0.001, 0.06)
ax[0].legend()


cl_mean_weighted = (2 * ell + 1) * cl_mean
ax[1].plot(ells, (2*ells +1)*cl_mean,
          marker="o", linestyle="--", color='cyan', alpha=0.6, lw=2.5,label=r"$ (2 \ell +1) \langle C_{\ell} \rangle$ Subsample N="+str(N))
ax[1].fill_between(ell, (2*ells +1)*(cl_mean - cl_std), (2*ells +1)*(cl_mean + cl_std), color='grey', alpha=0.2, label=r"$\pm 1 \sigma$ not scaled ")
ax[1].errorbar(ell, (2 * ell + 1)*cl_mean , yerr=(2 * ell + 1) *cl_std , fmt='o-', color='blue', alpha=0.2,
             ecolor='black', capsize=3, label=r"by $(2\ell +1)$")

ax[1].plot(ells, (2*ells +1)*cl_simu70_sub[0][:lmax + 1], 
         label="$(2 \ell +1) C_{\ell}$ Simu ", marker="o", linestyle="--", color='red', alpha=0.7)

ax[1].set_xlabel('$\ell$')
ax[1].set_xlim(-1, 20)
ax[1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[1].set_ylim(-0.008, 0.3)
ax[1].legend()

plt.tight_layout()
plt.show()



In [ ]:

fig, ax = plt.subplots(1,2, figsize=(8,4))
fig.suptitle('Simulation: Power Spectra,  ('+str(radio)+') kpc', fontsize=16)

ells = np.arange(lmax + 1)


ax[0].plot(ells, cl_mean, marker="o",  color='magenta', alpha=0.5,lw=2.5,label=r"$\langle C_{\ell} \rangle $ subsample N=100")
ax[0].fill_between(ell, cl_mean - cl_std, cl_mean + cl_std, color='grey', alpha=0.1, label=r"$\pm 1\sigma$")
#ax[0].bar(ell, cl_mean, yerr=cl_std, align='center', alpha=0.1, ecolor='black', capsize=1)
# Solo los puntos con barra de error
ax[0].errorbar(ell, cl_mean, yerr=cl_std, fmt='o-', color='orange',alpha=0.2,
             ecolor='black', capsize=3, label=r"")
ax[0].set_xlabel(r'$\ell$')
ax[0].set_xlim(-1, 15)
ax[0].set_ylabel(r'$C_{\ell}$')
ax[0].set_ylim(-0.001, 0.06)
ax[0].legend()


cl_mean_weighted = (2 * ell + 1) * cl_mean
cl_std_weighted = (2 * ell + 1) * cl_std

ax[1].plot(ell, cl_mean_weighted,color='cyan',alpha=0.7,marker='o',label=r"$\langle C_{\ell} \rangle $ Subsample N=100")
ax[1].fill_between(ell, cl_mean_weighted - cl_std_weighted,
                        cl_mean_weighted + cl_std_weighted,
                        color='grey', alpha=0.1, lw=2.5, label=r"$\pm 1\sigma$ scaled ")
ax[1].errorbar(ell, cl_mean_weighted , yerr=cl_std_weighted , fmt='o-', color='blue', alpha=0.2,
             ecolor='black', capsize=3, label=r"by $(2\ell +1)$")
ax[1].set_xlabel(r'$\ell$')
ax[1].set_ylabel(r'$(2 \ell +1) C_{\ell}$')
ax[1].legend()
ax[1].set_xlim(-1, 15)
ax[1].set_ylim(-0.008, 0.3)
plt.tight_layout()
plt.show()


## Para el cálculo del chi2

In [ ]:
print(len(cl_simu70[0]), len(cl_mean), len(cl_std))

In [ ]:
#cl:simu70 espectro 'observado' 
#cl_mean espectro modelo, la prediccion en base a los submuestreos
# cl_std incertidumbre de este modelo 

#  chi2
chi2 = np.sum((cl_simu70[0] - cl_mean[0])**2 / cl_std[0]**2)

# Grados de libertad (num de multipolos - 1)
dof = len(cl_mean) - 1

# Chi2 reducido
chi2_red = chi2 / dof


print("Chi-cuadrado total: ", chi2)
print("Chi-cuadrado reducido: ", chi2_red)


# Guardar todos los Cls en un df:

In [ ]:
# Escalado (2l + 1)
ell = ells  # array de multipolos

cl_mean_scaled  = (2 * ell + 1) * cl_mean
cl_simu70_scaled = (2 * ell + 1) * cl_simu70[0]
cl_simu_scaled = (2 * ell + 1) * cl_simu[0]
cl_std_scaled = (2 * ell + 1) * cl_std

# df
df_cls_simu = pd.DataFrame({
    'ell': ell,
    'cl_mean': cl_mean,
    'cl_std': cl_std,
    'cl_simu70': cl_simu70[0],
    'cl_simu': cl_simu[0],
    'cl_mean_scaled': cl_mean_scaled,
    'cl_simu70_scaled': cl_simu70_scaled,
    'cl_simu_scaled': cl_simu_scaled,
    'cl_std_scaled': cl_std_scaled,
})

# Guardar CSV
df_cls_simu.to_csv(f'cl_completo_con_escala_radio{radio}.csv', index=False)


### Leyendo el archivo para probar

In [ ]:

df_cls_simu = pd.read_csv('cl_completo_con_escala_radio70.csv')  


In [ ]:
df_cls_simu